In [1]:
import pandas as pd
import json
import os
import ast
import re
import numpy as np
import matplotlib.pyplot as plt
import torch
from time import time
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score, f1_score,  recall_score, confusion_matrix, precision_score
from transformers import (
    set_seed,
)
set_seed(42)

from unsloth import FastLanguageModel
max_seq_length = 500
from tqdm import tqdm

import pickle

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
random_state_list = [42, 57, 120, 98, 65, 74]

In [3]:
if os.getcwd() == '/root':
    new_path = "/root/0_Thesis/0_final/"
    os.chdir(new_path)
else:
    os.chdir("..") 
print(os.getcwd())

/root/0_Thesis/0_final


# Section 1: Load Human Label Data

In [28]:
df_evaluation = pd.read_csv("data/human/1_combine_hate_ds.csv")
df_evaluation['len_text'] = df_evaluation['text'].str.len()
df_evaluation = df_evaluation[df_evaluation['len_text'] <= 300]
print(df_evaluation.shape)

(84083, 7)


In [29]:
df_evaluation.dataset.value_counts()

dataset
ViHSD           30571
HateSpeechX     20022
Sexism          13631
GermEval2019    12131
GermEval2021     3457
Covid            2164
US_election      2107
Name: count, dtype: int64

In [30]:
df_evaluation.hate_label_id.value_counts()

hate_label_id
1    58915
0    25168
Name: count, dtype: int64

In [31]:
df_evaluation.language.value_counts()

language
eng    37924
vie    30571
deu    15588
Name: count, dtype: int64

In [32]:
df_evaluation = df_evaluation[df_evaluation['dataset'] == 'HateSpeechX']
df_evaluation['multi_label'] = 3
#df_hate_Discrimination,offensive,hate
df_evaluation.loc[df_evaluation['multi_label_id'] == '1', 'multi_label'] = 1
df_evaluation.loc[df_evaluation['multi_label_id'] == '2', 'multi_label'] = 2
df_evaluation.loc[df_evaluation['multi_label_id'] == '3', 'multi_label'] = 3


df_evaluation = df_evaluation.sample(frac=0.2, random_state=42)
df_evaluation['multi_label'].value_counts()

multi_label
3    1553
1    1367
2    1084
Name: count, dtype: int64

# Section 2: Load Model

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
model = "llama3.1"
model = "gemma2"

def chat_with_gemma2(messages, max_tokens=8000):
    chat = ChatOpenAI(
        openai_api_base="https://llms-inference.innkube.fim.uni-passau.de",
        api_key="",
        model=model,
        temperature=0.1
    )
    
    response = chat(messages=messages, max_tokens=max_tokens)
    return response.content


In [ ]:
prompt_template = """
You are tasked with annotating speech. Your response must be a single valid number corresponding to the category of the speech:

    1 for Hate Speech.
    2 for Offensive.
    3 for Normal.

Think step by step:
- Identify whether the comment contains explicit hate towards a group or individual based on identity characteristics (Hate Speech).
- If the comment is aggressive, rude, or contains insults but lacks clear hateful intent, classify it as Offensive Speech.
- If the comment is neutral or non-offensive, classify it as Normal Speech.
You will be given a list of comments. For each comment, return the number corresponding to the appropriate category. 

Provide **only** the numbers in a list format, with no explanations or additional text.

Your answer should be in this format:

[2, 1, 2, 3, 1]

Examples:
2. "You're an idiot! Learn how to think properly."  
1. "I hate all people of this group. They should be banned."  
3. "I hope you have a great day!"  
Your Answer: [2,1,3]  
Do you think the following comment is related to hate speech or offensive speech, or is it Normal speech?
1. "{comment_list}"

"""
df_evaluation = df_evaluation.reset_index()
df_evaluation = df_evaluation.sample(n=1000).reset_index()

# Section 3: Evaluation

In [36]:
def batch_comments(comments, batch_size):
    for i in range(0, len(comments), batch_size):
        yield comments[i:i + batch_size]
all_test = df_evaluation["text"].tolist()

result_text = []
batch_s = 5
col = f"{model}_few_short_5"
df_evaluation[col] = 3
counter = 0

for batch in tqdm(batch_comments(all_test, batch_s), desc="Processing batches", total=len(all_test)//batch_s + (1 if len(all_test)%batch_s != 0 else 0)):
    comment_list = "\n".join([f'{i+1}. "{comment}"' for i, comment in enumerate(batch)])
    
    final_prompt = prompt_template.format(comment_list=comment_list)
    
    messages = [
    SystemMessage(
        content="You are a helpful assistant."
    ),
    HumanMessage(
        content=final_prompt
    ),
    ]
    
    output = chat_with_gemma2(messages)
    try:
        int_list = ast.literal_eval(output)
    except:
        int_list = [3 for i in range(batch_s)]
    
    df_evaluation.loc[counter * batch_s : counter * batch_s + len(int_list) - 1 , col] = int_list[:batch_s]
    counter += 1
 
    result_text += int_list


Processing batches: 100%|█████████████████████████████████████████| 200/200 [16:05<00:00,  4.83s/it]


In [38]:
y_true = df_evaluation['multi_label']
y_pred = df_evaluation[col]
print("ACC: ",accuracy_score(y_true, y_pred))
print("Precision: ",precision_score(y_true, y_pred, average="macro"))
print("Recall: ",recall_score(y_true, y_pred, average="macro"))
print("F1: ",f1_score(y_true, y_pred, average="macro"))


ACC:  0.501
Precision:  0.5570869108976328
Recall:  0.5013171707489174
F1:  0.4567889340648998
